# Write Table From Scratch

In [ ]:
import tlc

PROJECT_NAME = "Write Table Sample"
DATASET_NAME = "Cat Images"
TABLE_NAME = "My Table"

root_url = tlc.Table.default_write_location() / PROJECT_NAME
table_writer = tlc.TableWriter(
    root_url=root_url,
    table_name=TABLE_NAME,
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
)

In [ ]:
# Helper function for creating bounding boxes
import random

MAX_LABEL = 10  # Define the maximum label value

def create_dummy_bounding_box():
    # Generate random center (x0, y0) and size (x1, y1) for the bounding box
    # This is yolo format: [label, x0, y0, x1, y1]
    x0, y0 = random.uniform(0.1, 0.9), random.uniform(0.1, 0.9)
    x1, y1 = random.uniform(0.05, 0.2), random.uniform(0.05, 0.2)
    
    # Generate a random label
    label = random.randint(0, MAX_LABEL)

    return {"x0": x0, "y0": y0, "x1": x1, "y1": y1, "label": label}

def create_dummy_bounding_boxes():
    # Generate a random number of bounding boxes between 3 and 5
    num_boxes = random.randint(3, 5)
    bounding_boxes = [create_dummy_bounding_box() for _ in range(num_boxes)]

    # Update image dimensions
    image_height = random.randint(100, 500)
    image_width = random.randint(100, 500)

    return {"image_height": image_height, "image_width": image_width, "bb_list": bounding_boxes}


In [ ]:
# Create table row by row
# Our table will contain four rows. Each row will have two columns: a image, and a bounding box.
table_rows = {
    "image": [],
    "bounding_boxes": []
}

for i in range(4):
    table_rows["image"].append(f"image_{i}.jpg")
    bounding_boxes = create_dummy_bounding_boxes()
    table_rows["bounding_boxes"].append(bounding_boxes)

In [ ]:
bb_schema = tlc.BoundingBoxListSchema(
    label_value_map={float(i): tlc.MapElement(f"label_{i}") for i in range(MAX_LABEL + 1)},
    x0_number_role=tlc.NUMBER_ROLE_BB_CENTER_X,
    y0_number_role=tlc.NUMBER_ROLE_BB_CENTER_Y,
    x1_number_role=tlc.NUMBER_ROLE_BB_SIZE_X,
    y1_number_role=tlc.NUMBER_ROLE_BB_SIZE_Y,
    x0_unit="relative",
    y0_unit="relative",
    x1_unit="relative",
    y1_unit="relative",
    include_segmentation=False,
)

schemas = {
    "image": tlc.Schema(value=tlc.StringValue(tlc.STRING_ROLE_IMAGE_URL)),
    "bounding_boxes": bb_schema
}

table_writer.set_override_column_schemas(override_column_schemas=schemas)

In [ ]:
table_writer.add_batch(table_rows)
table_writer.flush()

In [ ]:
infos = table_writer.get_written_table_infos()
print(infos)

In [ ]:
t = tlc.Table.from_url(infos[0]["url"])
t[0]